## **N-gram Frequency Analysis**

In [33]:
# Import packages
import pandas as pd
import numpy as np
from collections import Counter
import re
import ssl
# Fix SSL certificate issue on macOS/Anaconda
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

import nltk
nltk.download('wordnet', quiet=True)
from nltk.stem import WordNetLemmatizer

[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


In [34]:
# Set display options
pd.options.display.max_colwidth = 50
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format

In [35]:
# Load item data
file_items = '../data/meta_Home_and_Kitchen_filtered.csv'
df_items = pd.read_csv(file_items)

/var/folders/86/_khp3pb10vg5vtbr6fmndrxc0000gn/T/ipykernel_66965/2571311126.py:3: DtypeWarning: Columns (0: tech2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_items = pd.read_csv(file_items)


In [36]:
# Clean titles: lowercase, remove special characters, keep only letters, numbers, spaces
titles = df_items['title'].dropna().str.lower()
titles = titles.str.replace(r'[^a-z0-9\s]', '', regex=True)
titles = titles.str.strip()
print(f"Total titles: {len(titles)}")

Total titles: 1300506


## Text Cleaning: title, description, feature
Remove stopwords and lemmatize to create `title_1`, `description_1`, `feature_1`.

In [37]:
import ssl
# Fix SSL certificate issue on macOS/Anaconda
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

from nltk.corpus import stopwords

nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)

nltk_stop_words = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1028)>


In [44]:
def parse_list_string(text):
    """Convert string representation of a list into a single joined string.
    e.g. "['item1', 'item2']" -> 'item1 item2'
    Returns empty string for NaN or empty lists.
    """
    if pd.isna(text) or text == '[]':
        return ''
    try:
        items = ast.literal_eval(text)
        if isinstance(items, list):
            return ' '.join(str(item) for item in items)
    except (ValueError, SyntaxError):
        pass
    return str(text)


# Words to keep even though they are NLTK stopwords (useful for product features)
stopwords_to_keep = {'no', 'not', 'non', 'off', 'self'}
custom_stop_words = set(nltk_stop_words) - stopwords_to_keep


def clean_text(text):
    """Lowercase, remove special characters (keep letters, numbers, hyphens, decimals),
    remove stopwords (except negations), and lemmatize."""
    if pd.isna(text) or text == '':
        return ''

    text = text.lower()

    # 1. Standardize unit hyphens (e.g., "12-pc" -> "12 pc")
    text = re.sub(r'(\d+)-(pc|inch|oz|lb|piece)', r'\1 \2', text)

    # 2. Keep alphanumeric, spaces, decimals, and hyphens
    text = re.sub(r'[^a-z0-9\s.\-]', ' ', text)

    # 3. Keep dots between digits (10.5), remove others
    text = re.sub(r'(?<!\d)\.(?!\d)', ' ', text)

    # 4. Collapse spaces
    text = ' '.join(text.split())

    # 5. Tokenize, remove stopwords, and lemmatize
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in custom_stop_words]

    return ' '.join(tokens)

## Analysis for **Kitchen & Dining**

In [140]:
df_items_knd = df_items[df_items['cat_2'] == 'Kitchen & Dining']

In [141]:
# Apply clean_text to create title_1 in df_items_knd
df_items_knd = df_items_knd.copy()
df_items_knd['title_1'] = df_items_knd['title'].fillna('').apply(clean_text)


## Combined Analysis: Frequency + TF-IDF by Category (cat_3)
For each `cat_3`, show both the most common n-grams (raw frequency) and the most distinctive n-grams (TF-IDF). Frequency shows what's popular; TF-IDF shows what's unique to that category.

In [68]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Filter to valid cat_3 categories (more than 100 products to avoid noise)
cat3_counts = df_items_knd['cat_3'].value_counts()
valid_cat3 = cat3_counts[cat3_counts > 100].index.tolist()
df_knd_valid = df_items_knd[df_items_knd['cat_3'].isin(valid_cat3)]

# Concatenate all title_1 texts per cat_3 into a single document per category
cat3_docs = (
    df_knd_valid.groupby('cat_3')['title_1']
    .apply(lambda x: ' '.join(x.dropna()))
    .reset_index()
)
cat3_docs.columns = ['cat_3', 'text']

# Fit TF-IDF with unigrams, bigrams, and trigrams
tfidf = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=50000,
    min_df=1,
    sublinear_tf=True
)
tfidf_matrix = tfidf.fit_transform(cat3_docs['text'])
feature_names = tfidf.get_feature_names_out()

print(f"Valid cat_3 categories: {len(valid_cat3)}")
print(f"TF-IDF matrix: {tfidf_matrix.shape[0]} categories x {tfidf_matrix.shape[1]} n-grams")

Valid cat_3 categories: 14
TF-IDF matrix: 14 categories x 50000 n-grams


In [80]:
# Combined: Frequency + TF-IDF for each cat_3
top_n = 100

for idx, row in cat3_docs.iterrows():
    cat_name = row['cat_3']
    cat_count = cat3_counts[cat_name]
    df_cat = df_knd_valid[df_knd_valid['cat_3'] == cat_name]
    titles = df_cat['title_1'].dropna()
    all_words = ' '.join(titles).split()

    # --- Raw frequency ---
    unigram_counts = Counter(all_words)
    bigram_counts = Counter(zip(all_words, all_words[1:]))
    trigram_counts = Counter(zip(all_words, all_words[1:], all_words[2:]))

    # --- TF-IDF ---
    scores = tfidf_matrix[idx].toarray().flatten()
    top_tfidf_indices = scores.argsort()[::-1][:top_n]

    print(f"\n{'='*70}")
    print(f"cat_3: {cat_name} ({cat_count} products)")
    print(f"{'='*70}")

    # Frequency tables
    print(f"\n--- Top {top_n} Unigrams (Frequency) ---")
    for word, count in unigram_counts.most_common(top_n):
        print(f"  {word:30s} {count:>8,}")

    print(f"\n--- Top {top_n} Bigrams (Frequency) ---")
    for bg, count in bigram_counts.most_common(top_n):
        print(f"  {bg[0]} {bg[1]:30s} {count:>8,}")

    print(f"\n--- Top {top_n} Trigrams (Frequency) ---")
    for tg, count in trigram_counts.most_common(top_n):
        print(f"  {tg[0]} {tg[1]} {tg[2]:30s} {count:>8,}")

    # TF-IDF table
    print(f"\n--- Top {top_n} N-grams (TF-IDF) ---")
    print(f"  {'Rank':<6} {'TF-IDF':>8}  {'N-gram'}")
    print(f"  {'-'*6} {'-'*8}  {'-'*40}")
    for rank, i in enumerate(top_tfidf_indices, 1):
        if scores[i] > 0:
            print(f"  {rank:<6} {scores[i]:>8.4f}  {feature_names[i]}")


cat_3: Bakeware (39620 products)

--- Top 100 Unigrams (Frequency) ---
  mold                             11,777
  -                                 8,032
  cake                              7,640
  cookie                            7,244
  cutter                            7,222
  set                               6,662
  inch                              6,495
  silicone                          6,085
  pan                               5,941
  quot                              5,882
  chocolate                         5,556
  baking                            5,381
  candy                             5,001
  cupcake                           3,875
  2                                 3,410
  4                                 3,350
  steel                             3,240
  3                                 2,886
  cup                               2,708
  x                                 2,701
  piece                             2,426
  wilton                            2,415
  fo

In [156]:
import json

# Load metadata and global filters
with open('../data/master_metadata.json', 'r') as f:
    metadata = json.load(f)

with open('../data/global_filters.json', 'r') as f:
    global_filters = json.load(f)

# Sort global filters longest-first for greedy matching
global_filters = sorted(global_filters, key=len, reverse=True)


def remove_global_filters(text, filters):
    """Remove global filter terms from text using word-boundary-aware replacement."""
    if not text:
        return text
    for term in filters:
        pattern = r'\b' + re.escape(term) + r'\b'
        text = re.sub(pattern, '', text)
    # Collapse whitespace
    text = ' '.join(text.split())
    return text


def extract_features(title_text, cat3_name, meta):
    """Extract structured features from title text based on category metadata.

    Returns (features_dict, remaining_text).
    """
    features = {}

    if not title_text or pd.isna(cat3_name) or cat3_name not in meta:
        return (features, title_text if title_text else '')

    cat_meta = meta[cat3_name]

    for attr_name, attr_info in cat_meta.items():
        if attr_info['type'] == 'dictionary':
            # Sort values longest-first to avoid partial matches
            values = sorted(attr_info['values'], key=len, reverse=True)
            matched = []
            for val in values:
                pattern = r'\b' + re.escape(val) + r'\b'
                if re.search(pattern, title_text):
                    matched.append(val)
                    title_text = re.sub(pattern, '', title_text)
            if matched:
                features[attr_name] = matched

        elif attr_info['type'] == 'regex':
            matched = []
            for pat in attr_info['patterns']:
                match = re.search(pat, title_text)
                if match:
                    matched.append(match.group(0))
                    title_text = title_text[:match.start()] + title_text[match.end():]
            if matched:
                features[attr_name] = matched

    # Collapse whitespace
    title_text = ' '.join(title_text.split())
    return (features, title_text)


# Step A: Remove global filters from title_1, store in title_2
df_items_knd['title_2'] = df_items_knd['title_1'].apply(
    lambda text: remove_global_filters(text, global_filters)
)

# Step B: Extract features from title_2 → extracted_features + title_2 updated
results = df_items_knd.apply(
    lambda row: extract_features(row['title_2'], row['cat_3'], metadata),
    axis=1, result_type='expand'
)
df_items_knd['extracted_features'] = results[0]
df_items_knd['title_2'] = results[1]

# Step C: Re-run extraction on title_2 with updated metadata/filters → extracted_features1 + title_3
# Reload to pick up any changes made to the files
# with open('../data/master_metadata.json', 'r') as f:
#     metadata = json.load(f)
# with open('../data/global_filters.json', 'r') as f:
#     global_filters = json.load(f)
# global_filters = sorted(global_filters, key=len, reverse=True)
#
# # Remove global filters from title_2
# df_items_knd['title_3'] = df_items_knd['title_2'].apply(
#     lambda text: remove_global_filters(text, global_filters)
# )
#
# # Extract features from title_3 → extracted_features1 + title_3 updated
# results2 = df_items_knd.apply(
#     lambda row: extract_features(row['title_3'], row['cat_3'], metadata),
#     axis=1, result_type='expand'
# )
# df_items_knd['extracted_features1'] = results2[0]
# df_items_knd['title_3'] = results2[1]
#
# # --- Verification ---
# print(f"Rows with extracted_features:  {(df_items_knd['extracted_features'].apply(len) > 0).sum()} / {len(df_items_knd)}")
# print(f"Rows with extracted_features1: {(df_items_knd['extracted_features1'].apply(len) > 0).sum()} / {len(df_items_knd)}")
# print()
#
# # Show a few examples per category
# for cat3 in sorted(df_items_knd['cat_3'].dropna().unique()):
#     if cat3 not in metadata:
#         continue
#     sample = df_items_knd[
#         (df_items_knd['cat_3'] == cat3) &
#         (df_items_knd['extracted_features1'].apply(len) > 0)
#     ].head(3)
#     if len(sample) == 0:
#         continue
#     print(f"--- {cat3} ---")
#     for _, row in sample.iterrows():
#         print(f"  title_1:  {row['title_1'][:80]}")
#         print(f"  title_2:  {row['title_2'][:80]}")
#         print(f"  title_3:  {row['title_3'][:80]}")
#         print(f"  features: {row['extracted_features']}")
#         print(f"  features1:{row['extracted_features1']}")
#         print()


Rows with extracted_features:  369668 / 448139
Rows with extracted_features1: 8141 / 448139

--- Bakeware ---
  title_1:  chicago metallic professional nonstick cookie sheet 15.75 inch-by-13.75 inch
  title_2:  chicago metallic professional nonstick cookie sheet -by-13.75 inch
  title_3:  chicago metallic professional nonstick cookie sheet -by-
  features: {'Dimensions': ['15.75 inch']}
  features1:{'Dimensions': ['13.75 inch']}

  title_1:  chicago metallic professional nonstick 13 1 2 inch 9 1 2 inch cookie sheet
  title_2:  chicago metallic professional nonstick 13 1 9 1 2 inch cookie sheet
  title_3:  chicago metallic professional nonstick 13 1 9 1 cookie sheet
  features: {'Dimensions': ['2 inch']}
  features1:{'Dimensions': ['2 inch']}

  title_1:  frieling usa brotform oval bread rising basket 10 inch 7 inch
  title_2:  frieling usa brotform oval bread rising basket 7 inch
  title_3:  frieling usa brotform oval bread rising basket
  features: {'Dimensions': ['10 inch']}
  featur

### **Check the outcomes**

In [163]:
df_items_knd1 = df_items_knd.copy()
df_items_knd1['title_1'] = df_items_knd1['title'].fillna('').apply(clean_text)


In [164]:
import json

# Load metadata and global filters
with open('../data/master_metadata.json', 'r') as f:
    metadata = json.load(f)
with open('../data/global_filters.json', 'r') as f:
    global_filters = json.load(f)
global_filters = sorted(global_filters, key=len, reverse=True)

# Step A: Remove global filters from title_1 → title_2
df_items_knd1['title_2'] = df_items_knd1['title_1'].apply(
    lambda text: remove_global_filters(text, global_filters)
)

# Step B: Extract features from title_2 → extracted_features + update title_2
results = df_items_knd1.apply(
    lambda row: extract_features(row['title_2'], row['cat_3'], metadata),
    axis=1, result_type='expand'
)
df_items_knd1['extracted_features'] = results[0]
df_items_knd1['title_2'] = results[1]

# Verification
print(f"Rows with extracted features: {(df_items_knd1['extracted_features'].apply(len) > 0).sum()} / {len(df_items_knd1)}")
print()
for cat3 in sorted(df_items_knd1['cat_3'].dropna().unique()):
    if cat3 not in metadata:
        continue
    sample = df_items_knd1[
        (df_items_knd1['cat_3'] == cat3) &
        (df_items_knd1['extracted_features'].apply(len) > 0)
    ].head(3)
    if len(sample) == 0:
        continue
    print(f"--- {cat3} ---")
    for _, row in sample.iterrows():
        print(f"  title_1: {row['title_1'][:80]}")
        print(f"  title_2: {row['title_2'][:80]}")
        print(f"  features: {row['extracted_features']}")
        print()

Rows with extracted features: 370025 / 448139

--- Bakeware ---
  title_1: pink stripe baking cup
  title_2: stripe
  features: {'Product_Type': ['baking cup'], 'Color': ['pink']}

  title_1: toot sweet cupcake kit - pink
  title_2: toot sweet cupcake kit -
  features: {'Color': ['pink']}

  title_1: nutcracker cupcake kit - 48 piece set
  title_2: nutcracker cupcake kit - set
  features: {'Piece_Count': ['48 piece']}

--- Coffee, Tea & Espresso ---
  title_1: sm aring land b ouml na ikea - anrik coffee tea maker stainless steel
  title_2: sm land b na ikea - anrik coffee
  features: {'Product_Type': ['tea maker'], 'Material': ['stainless steel']}

  title_1: japanese tea ceremony matcha bowl scoop whisk gift box set naruto design
  title_2: japanese tea ceremony whisk set naruto
  features: {'Sub_Type': ['matcha bowl'], 'Features': ['scoop']}

  title_1: japanese ginsugi tea ceremony matcha bowl scoop whisk gift box set
  title_2: japanese ginsugi tea ceremony whisk set
  features: {'

In [173]:
# df_items_knd1[['title', 'title_1', 'title_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'extracted_features']].iloc[20:35,]
df_items_knd1[['title', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'extracted_features']]

,title,cat_3,cat_4,cat_5,cat_6,extracted_features
0,You Are Special Today Red Plate [With Red Pen],Dining & Entertaining,Dinnerware,Plates,Dinner Plates,{'Color': ['red']}
2,Artistic Churchware Communion Cup Filler: RW525,Dining & Entertaining,Glassware & Drinkware,Wine & Champagne Glasses,NaN,{}
10,"Insulated Lunch Bag, Lunchbox, Bento Box (Pack...",Storage & Organization,Travel & To-Go Food Containers,Lunch Bags,NaN,"{'Product_Type': ['bento box'], 'Color': ['red..."
14,Gulf Coast Kitchen Seafood By Pam Johnson,Kitchen Utensils & Gadgets,NaN,NaN,NaN,{}
15,Gallopade Publishing Group Alabama Flag Magnet...,Kitchen Utensils & Gadgets,Kitchen Accessories,Refrigerator Magnets,NaN,{'Product_Type': ['magnet']}
...,...,...,...,...,...,...
1300533,Crock Pot 114311.01RM Denhoff 8.25&quot; Ribbe...,Bakeware,Bakers & Casseroles,Casseroles,NaN,"{'Product_Type': ['casserole'], 'Color': ['red..."
1300534,Gibson Home 108157.04 Reilly 4 Piece Non-Stick...,Cookware,All Pans,NaN,NaN,"{'Color': ['black'], 'Features': ['non-stick']..."
1300535,GE WR32X10594 Cover Pan Glass,Small Appliance Parts & Accessories,NaN,NaN,NaN,"{'Brand': ['ge'], 'Material': ['glass']}"
1300536,Bosch 00675261 Ice Maker,Small Appliances,NaN,NaN,NaN,{}


In [182]:
df_items_knd1[df_items_knd1['cat_3'] == 'Dining & Entertaining'].sample(n=100, random_state=42)

,category,tech1,description,title,tech2,brand,feature,rank,main_cat,price,asin,date,description_parsed,feature_parsed,title_1,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,title_clean,title_2,extracted_features,title_3,extracted_features1
1266315,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,"['MiiR is a socially responsible company, help...",MiiR 16 Oz Insulated Pint Glass with BRBC logo,NaN,Bear Republic Brewing Company,"[""16 oz, double wall vacuum insulated (doesn't...","['>#890,438 in Kitchen & Dining (See Top 100 i...",Amazon Home,$20.00,B01FLCRDHY,"May 13, 2016","MiiR is a socially responsible company, helpin...","16 oz, double wall vacuum insulated (doesn't s...",miir 16 oz insulated pint glass brbc logo,Home & Kitchen,Kitchen & Dining,Dining & Entertaining,Glassware & Drinkware,Beer Glasses,NaN,miir 16 oz insulated pint glass with brbc logo,miir brbc logo,"{'Product_Type': ['pint glass'], 'Features': [...",miir brbc logo,{}
217806,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,['The Woozie is a wine coozie that is fun and ...,Bling Woozie Party Pack By Overstock Outdoors ...,NaN,Woozie,"['Wine glass design', 'Fun and functional', 'T...","['>#1,481,976 in Patio, Lawn & Garden (See Top...",Amazon Home,NaN,B003TTXQTQ,NaN,The Woozie is a wine coozie that is fun and fu...,Wine glass design Fun and functional This uniq...,bling woozie party pack overstock outdoors store,Home & Kitchen,Kitchen & Dining,Dining & Entertaining,Bar Tools & Drinkware,NaN,NaN,bling woozie party pack by overstock outdoors ...,bling woozie party pack overstock outdoors store,{},bling woozie party pack overstock outdoors store,{}
37887,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,"[""Store your butter and margarine in a special...",Bianca White Porcelain and Stainless Steel 11 ...,NaN,Trudeau,['Set your table with a butter/margarine dish'...,"['>#1,480,067 in Kitchen & Dining (See Top 100...",Amazon Home,NaN,B000A7DAS2,"November 2, 2006",Store your butter and margarine in a special d...,Set your table with a butter/margarine dish .7...,bianca white porcelain stainless steel 11 ounc...,Home & Kitchen,Kitchen & Dining,Dining & Entertaining,Serveware,Butter Dishes,NaN,bianca white porcelain and stainless steel 11 ...,bianca butter margarine dish,"{'Material': ['stainless steel', 'porcelain'],...",bianca butter margarine dish,{}
1117741,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,"['16oz Latte Stoneware Mug with fun saying: ""I...",Sweet Bird and Company LM004 Latte Stoneware M...,NaN,Sweet Bird and Company,"['Saying: ""I\'ll have a cafe-mocha-vodka-valiu...","['>#759,855 in Kitchen & Dining (See Top 100 i...",Amazon Home,NaN,B0171B0YLA,"October 20, 2015","16oz Latte Stoneware Mug with fun saying: ""I'l...","Saying: ""I'll have a cafe-mocha-vodka-valium-l...",sweet bird company lm004 latte stoneware mug 1...,Home & Kitchen,Kitchen & Dining,Dining & Entertaining,Dinnerware,"Cups, Mugs, & Saucers",Coffee Cups & Mugs,sweet bird and company lm004 latte stoneware m...,sweet bird company lm004 latte,"{'Product_Type': ['mug'], 'Material': ['stonew...",sweet bird company lm004 latte,{}
703740,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,"['glass tumblers, set of 4']",Anchor Hocking Newport Small Glass Tumblers ~ ...,NaN,Anchor Hocking,"['clear', 'glass', 'tumbler']","['>#481,739 in Kitchen & Dining (See Top 100 i...",Amazon Home,$15.90,B00IK8GB7A,"February 21, 2014","glass tumblers, set of 4",clear glass tumbler,anchor hocking newport small glass tumbler set 4,Home & Kitchen,Kitchen & Dining,Dining & Entertaining,Glassware & Drinkware,Tumblers & Water Glasses,NaN,anchor hocking newport small glass tumblers s...,anchor hocking newport small tumbler,"{'Material': ['glass'], 'Piece_Count': ['set 4']}",anchor hocking newport small tumbler,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
541074,"['Home & Kitchen', 'Kitchen & Dining', 'Dining...",NaN,"['Wok, 13 / 3

In [183]:
# Sample 100 random rows and merge fields into a list for LLM scoring
sample_100 = df_items_knd1[df_items_knd1['cat_3'] == 'Dining & Entertaining'].sample(n=30, random_state=42)

llm_input = []
for _, row in sample_100.iterrows():
    item = {
        'title': row['title'] if pd.notna(row['title']) else '',
        'cat_3': row['cat_3'] if pd.notna(row['cat_3']) else '',
        'cat_4': row['cat_4'] if pd.notna(row['cat_4']) else '',
        'cat_5': row['cat_5'] if pd.notna(row['cat_5']) else '',
        'cat_6': row['cat_6'] if pd.notna(row['cat_6']) else '',
        'extracted_features': row['extracted_features']
    }
    llm_input.append(item)

print(f"Total items: {len(llm_input)}")
llm_input[:3]

Total items: 30


[{'title': 'MiiR 16 Oz Insulated Pint Glass with BRBC logo',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Glassware & Drinkware',
  'cat_5': 'Beer Glasses',
  'cat_6': '',
  'extracted_features': {'Product_Type': ['pint glass'],
   'Features': ['insulated'],
   'Capacity_Volume': ['16 oz']}},
 {'title': 'Bling Woozie Party Pack By Overstock Outdoors Store',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Bar Tools & Drinkware',
  'cat_5': '',
  'cat_6': '',
  'extracted_features': {}},
 {'title': 'Bianca White Porcelain and Stainless Steel 11 ounce Butter and Margarine Dish',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Serveware',
  'cat_5': 'Butter Dishes',
  'cat_6': '',
  'extracted_features': {'Material': ['stainless steel', 'porcelain'],
   'Color': ['white'],
   'Capacity_Volume': ['11 ounce']}}]

In [184]:
llm_input

[{'title': 'MiiR 16 Oz Insulated Pint Glass with BRBC logo',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Glassware & Drinkware',
  'cat_5': 'Beer Glasses',
  'cat_6': '',
  'extracted_features': {'Product_Type': ['pint glass'],
   'Features': ['insulated'],
   'Capacity_Volume': ['16 oz']}},
 {'title': 'Bling Woozie Party Pack By Overstock Outdoors Store',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Bar Tools & Drinkware',
  'cat_5': '',
  'cat_6': '',
  'extracted_features': {}},
 {'title': 'Bianca White Porcelain and Stainless Steel 11 ounce Butter and Margarine Dish',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Serveware',
  'cat_5': 'Butter Dishes',
  'cat_6': '',
  'extracted_features': {'Material': ['stainless steel', 'porcelain'],
   'Color': ['white'],
   'Capacity_Volume': ['11 ounce']}},
 {'title': 'Sweet Bird and Company LM004 Latte Stoneware Mug, 16 oz, White',
  'cat_3': 'Dining & Entertaining',
  'cat_4': 'Dinnerware',
  'cat_5': 'Cups, Mugs, & Saucers',
  'ca

## **Analysis for Home Decor**

In [197]:
# Filter to Home Decor and clean titles
df_items_hd = df_items[df_items['cat_2'] == 'Home Dcor'].copy()
df_items_hd['title_1'] = df_items_hd['title'].fillna('').apply(clean_text)
print(f"Total Home Decor items: {len(df_items_hd)}")
print(f"cat_3 categories: {df_items_hd['cat_3'].nunique()}")
df_items_hd['cat_3'].value_counts()

Total Home Decor items: 352389
cat_3 categories: 60


cat_3
Home Dcor Accents                                                                                                                                               158806
Candles & Holders                                                                                                                                                34389
Area Rugs, Runners & Pads                                                                                                                                        27269
Window Treatments                                                                                                                                                22075
Clocks                                                                                                                                                           22053
Picture Frames                                                                                                                                                 

In [198]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Filter to valid cat_3 categories (more than 100 products to avoid noise)
cat3_counts_hd = df_items_hd['cat_3'].value_counts()
valid_cat3_hd = cat3_counts_hd[cat3_counts_hd > 100].index.tolist()
df_hd_valid = df_items_hd[df_items_hd['cat_3'].isin(valid_cat3_hd)]

# Concatenate all title_1 texts per cat_3 into a single document per category
cat3_docs_hd = (
    df_hd_valid.groupby('cat_3')['title_1']
    .apply(lambda x: ' '.join(x.dropna()))
    .reset_index()
)
cat3_docs_hd.columns = ['cat_3', 'text']

# Fit TF-IDF with unigrams, bigrams, and trigrams
tfidf_hd = TfidfVectorizer(
    ngram_range=(1, 3),
    max_features=50000,
    min_df=1,
    sublinear_tf=True
)
tfidf_matrix_hd = tfidf_hd.fit_transform(cat3_docs_hd['text'])
feature_names_hd = tfidf_hd.get_feature_names_out()

print(f"Valid cat_3 categories: {len(valid_cat3_hd)}")
print(f"TF-IDF matrix: {tfidf_matrix_hd.shape[0]} categories x {tfidf_matrix_hd.shape[1]} n-grams")

Valid cat_3 categories: 21
TF-IDF matrix: 21 categories x 50000 n-grams


In [199]:
# Combined: Frequency + TF-IDF for each cat_3
top_n = 100

for idx, row in cat3_docs_hd.iterrows():
    cat_name = row['cat_3']
    cat_count = cat3_counts_hd[cat_name]
    df_cat = df_hd_valid[df_hd_valid['cat_3'] == cat_name]
    titles = df_cat['title_1'].dropna()
    all_words = ' '.join(titles).split()

    # --- Raw frequency ---
    unigram_counts = Counter(all_words)
    bigram_counts = Counter(zip(all_words, all_words[1:]))
    trigram_counts = Counter(zip(all_words, all_words[1:], all_words[2:]))

    # --- TF-IDF ---
    scores = tfidf_matrix_hd[idx].toarray().flatten()
    top_tfidf_indices = scores.argsort()[::-1][:top_n]

    print(f"\n{'='*70}")
    print(f"cat_3: {cat_name} ({cat_count} products)")
    print(f"{'='*70}")

    # Frequency tables
    print(f"\n--- Top {top_n} Unigrams (Frequency) ---")
    for word, count in unigram_counts.most_common(top_n):
        print(f"  {word:30s} {count:>8,}")

    print(f"\n--- Top {top_n} Bigrams (Frequency) ---")
    for bg, count in bigram_counts.most_common(top_n):
        print(f"  {bg[0]} {bg[1]:30s} {count:>8,}")

    print(f"\n--- Top {top_n} Trigrams (Frequency) ---")
    for tg, count in trigram_counts.most_common(top_n):
        print(f"  {tg[0]} {tg[1]} {tg[2]:30s} {count:>8,}")

    # TF-IDF table
    print(f"\n--- Top {top_n} N-grams (TF-IDF) ---")
    print(f"  {'Rank':<6} {'TF-IDF':>8}  {'N-gram'}")
    print(f"  {'-'*6} {'-'*8}  {'-'*40}")
    for rank, i in enumerate(top_tfidf_indices, 1):
        if scores[i] > 0:
            print(f"  {rank:<6} {scores[i]:>8.4f}  {feature_names_hd[i]}")


cat_3: Area Rugs, Runners & Pads (27269 products)

--- Top 100 Unigrams (Frequency) ---
  rug                              31,491
  area                             14,287
  x                                13,630
  quot                             13,205
  3                                 6,785
  5                                 5,824
  6                                 5,652
  inch                              5,146
  2                                 5,143
  collection                        4,954
  7                                 4,534
  8                                 4,133
  -                                 3,986
  runner                            3,951
  carpet                            3,579
  10                                3,475
  blue                              2,954
  4                                 2,726
  black                             2,439
  home                              2,281
  9                                 2,267
  foot                       

In [201]:
# Check if any cat_3 belongs to more than one cat_2
cat3_cat2 = df_items.groupby('cat_3')['cat_2'].nunique()
duplicates = cat3_cat2[cat3_cat2 > 1]
if len(duplicates) > 0:
    print("cat_3 values with multiple cat_2 parents:")
    for cat3 in duplicates.index:
        counts = df_items[df_items['cat_3'] == cat3].groupby('cat_2').size()
        print(f"\n  {cat3}:")
        for cat2, count in counts.items():
            print(f"    {cat2}: {count:,}")
else:
    print("All cat_3 values belong to exactly one cat_2.")

cat_3 values with multiple cat_2 parents:
  10" high: ['Furniture', 'Wall Art', 'Event & Party Supplies']
  100% Cotton: ['Kitchen & Dining', 'Bath']
  100% Polyester: ['Event & Party Supplies', 'Seasonal Dcor', 'Storage & Organization', 'Kitchen & Dining', 'Bath']
  </span></span></span>: ['Furniture', 'Kitchen & Dining', 'Wall Art']
  Imported: ['Event & Party Supplies', 'Home Dcor', 'Kitchen & Dining', 'Storage & Organization', 'Seasonal Dcor', 'Bedding', 'Bath']
  Made in USA: ['Home Dcor', 'Storage & Organization']
  Not Applicable: ['Event & Party Supplies', 'Seasonal Dcor']
  Patchwork: ['Bedding', 'Bath']
  Polyester: ['Storage & Organization', 'Event & Party Supplies']
